In [2]:
#Import statements
import pandas as pd
from pandas import datetime as dt
import csv
import numpy as np
#import lib.google_sheet_api as gs
import os

/var/folders/0_/q6d4fymj0v9bl3m8gdcnl6n40000gn/T/ipykernel_40781/1883485394.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime as dt


In [3]:
filename = "/Users/suneetakartha/Downloads/Med Consumption.csv"

In [4]:
#Get data

# site_interval_data = pd.read_csv(filename)
site_interval_data.columns = ['date', 'month', 'day', 'hour', 'load_net_solar', 'solar', 'battery','gross_load','soc_real','soc_sterile_2hr','soc_sterile_4hr']


site_interval_data['date']=pd.to_datetime(site_interval_data['date'], utc=True)
site_interval_data['solar']=site_interval_data['solar'].fillna(0)

site_interval_data['week'] = site_interval_data['date'] - pd.offsets.Week(weekday=6)
site_interval_data['dow']= site_interval_data['date'].dt.dayofweek
site_interval_data['month']= site_interval_data['date'].dt.month

d=({'load_net_solar':'load_net_solar.sum', 'solar':'solar.sum', 'battery':'battery.sum','gross_load':'gross_load.sum'
                    ,'soc_real':'soc_real.avg','soc_sterile_2hr':'soc_sterile_2hr.avg','soc_sterile_4hr':'soc_sterile_4hr.avg'})
site_interval_data=(site_interval_data.groupby(['date','month', 'hour'])
                    .agg({'load_net_solar':'sum', 'solar':'sum', 'battery':'sum','gross_load':'sum'
                          ,'soc_real':'mean','soc_sterile_2hr':'mean','soc_sterile_4hr':'mean'})).reset_index()
# site_interval_data['SOC_net']= site_interval_data.groupby(['date'])['solar'].cumsum()

In [75]:
xls = pd.ExcelFile('~/Downloads/ELRP_sample_data.xlsx')
site_interval_data1 = pd.read_excel(xls, 'Battery Scenario 1.1.A')
site_interval_data2 = pd.read_excel(xls, 'Battery Scenario 1.2.A')
site_interval_data3 = pd.read_excel(xls, 'Battery Scenario 1.3.A')
site_interval_data4 = pd.read_excel(xls, 'Battery Scenario 2.1.A')
site_interval_data5 = pd.read_excel(xls, 'Battery Scenario 2.2.A')
site_interval_data6 = pd.read_excel(xls, 'Battery Scenario 2.3.A')

def clean_data(site_interval_data):
    site_interval_data.columns = ['date', 'hour', 'month', 'gross_load', 'battery', 'battery_2_hr', 'battery_4_hr']


    site_interval_data['date']=pd.to_datetime(site_interval_data['date'], utc=True)

    site_interval_data['week'] = site_interval_data['date'] - pd.offsets.Week(weekday=6)
    site_interval_data['dow']= site_interval_data['date'].dt.dayofweek
    site_interval_data['month']= site_interval_data['date'].dt.month

    d=({'gross_load':'gross_load.sum', 'battery':'battery.sum','battery_2_hr':'battery_2_hr.sum'
                        ,'battery_4_hr':'battery_4_hr.avg'})
    site_interval_data=(site_interval_data.groupby(['date','month', 'hour'])
                        .agg({'gross_load':'sum', 'battery':'sum', 
                              'battery_2_hr':'sum','battery_4_hr':'sum'})).reset_index()

    return site_interval_data

In [76]:
df6= clean_data(site_interval_data6)

In [77]:
test6=get_weekly_baseline(df6,0,16,21)

In [78]:
elrp6= get_revenue_values(test6)
elrp6.to_csv('~/Downloads/ELRP_Generac8.csv')

In [68]:

def get_weekly_baseline(x, day_of_bl, hour_start, hour_end):
    x['dow']= x['date'].dt.dayofweek
    x['week'] = x['date'] - pd.offsets.Week(weekday=6)

#     weekday_series= x['dow']
    ## Limit data set to the hours in question
    
    #NOTES: things to consider going forward:
    # 1) rotating the days based on historic load consumption
    # 2) rolling 45 days 
    moo_hours= x[(x['hour']>= hour_start) & (x['hour']<hour_end)]
    
# The below code takes: 1) Rolling 45 days 2) Find and average the five highest days IF that day 
# is the dow specified in the function 
    
    
    moo_hours['baseline_setting_base']=np.where(moo_hours['dow']==0, moo_hours['gross_load'], 0)
    moo_hours['best_hour_kw'] = moo_hours.groupby(['hour'])['baseline_setting_base'].transform(lambda x:x.rolling(45).apply(lambda x: np.mean(sorted(x,reverse=True)[:5])))
#     moo_hours['array'] = moo_hours.groupby(['hour'])['baseline_setting_base'].transform(lambda x:x.rolling(45).apply(lambda x: np.mean(sorted(x,reverse=True)[:5])))
#     moo_hours['dummy'] = moo_hours.groupby('firm')['baseline_setting_base'].transform(lambda x: x.rolling(45).max().le(moo_hours['best_hour_kw']))

    
    z=moo_hours[moo_hours['dow']==0]
    z=z[['week','hour','best_hour_kw']]
    
    #     x.set_index('hour')
#     x['rolling_average']= x.groupby(['hour'])['gross_load'].transform(lambda x: np.mean(sorted(x,reverse=True)[:5]))
    
            
    moo_hours = moo_hours.merge(right = z, on = ['week', 'hour'])
       
    moo_hours=moo_hours.drop(columns=['best_hour_kw_x'])
    moo_hours=moo_hours.sort_values(['date','hour'])
    moo_hours=moo_hours.drop_duplicates()
#     x = x.drop(columns = ['solar_x','unit_x','genset_x','genset_y', 'date_y', 'gross_load_y','solar_y', 'storage_y', 'net load_x','unit_y', 'soc_y','dow_y'])
#     x = x.rename(columns={'date_x': 'date','gross_load_x':'gross_load', 'storage_x':'storage', 'net load_y':'net_load', 'soc_x':'soc'})
    ##redo bl calc for weekends 

    return moo_hours


In [70]:
def get_revenue_values(test):
    test['month']= test['date'].dt.month
    test['day']= test['date'].dt.day
    test['year']= test['date'].dt.year
    min_month= (test.groupby(['month','year']).agg({'day':'min',
                                             'hour':'min',
                             'baseline_setting_base': lambda x: (x > 0).mean()})).reset_index()

    data = test.merge(min_month, on = ['year','month','day', 'hour'], how='left')
    data= data.fillna(0)

    data

    _2022_cap_rate=({'month': [1,2,3,4,5,6,7,8,9,10,11,12], 
                     'value':[2970, 2963, 2999, 3037, 5036, 5368, 5472, 5474, 5479, 5298, 2778, 2684]})

    idktest= pd.DataFrame.from_dict(_2022_cap_rate)
    idktest
    # idktest = idktest.rename(columns=('month', 'value'))
    data2 = data.merge(idktest, on = ['month'], how='left')

    data2['cap_rev']= data2['baseline_setting_base_y']* data2['value']
    data2

    energy_rev_data=pd.read_csv('/Users/suneetakartha/Downloads/energy_revenue_sizing_data.csv')
    energy_rev_data =pd.melt(energy_rev_data, id_vars=['Month','Day'], value_vars=['17', '18','19','20'], ignore_index=False)
    energy_rev_data.columns=['month', 'day', 'hour', 'energy_rev_val']
    energy_rev_data['hour']=energy_rev_data['hour'].astype('int64')
    final_rev= data2.merge(energy_rev_data, on = ['month','day','hour'], how='left')
    final_rev['energy_rev']=np.where(final_rev['baseline_setting_base_x']>0, 0, final_rev['best_hour_kw_y']/1000* final_rev['energy_rev_val'])
    final_rev=final_rev.drop(columns=['baseline_setting_base_y','value', 'energy_rev_val'])
    return final_rev

In [6]:
def get_10in10_baseline_unadj(x): 
    #x must be a data frame with the following columns:
    #datetime -- a date time column in pd, in utc
    #load
    #x = incremental.copy()
#     x['date'] = x['date'].dt.strftime("%Y-%m-%d")
#     weekday_series =  x['date'].dt.dayofweek
#     x['is_weekday'] = (weekday_series!=6) & (weekday_series!=5)
    x['time_id']=x['hour']
    

    date = x['date']
    time_id = x['time_id']
#     is_weekday = x['is_weekday']
    load = x['gross_load']

    dt = pd.DataFrame({'datetime':x['date'],
                       'date':date, 
                       'time_id':time_id, 
#                        'is_weekday': is_weekday, 
                       'gross_load':load})
    grouped = dt.groupby(['time_id'])
    result = pd.DataFrame()
    for name, group in grouped:
        group = group.sort_values('date', ascending = True)
        group['baseline_unadj'] = group['gross_load'].rolling(window=5, 
                                                       min_periods=10).mean().shift()
        result = pd.concat([result,group])
    x = x.merge(right = result, on = ['date', 'time_id', 'gross_load'])
    x = x.drop(columns = ['date'])
    return x

In [48]:
test2.to_csv('~/Downloads/Resi_ELRP_GroupB.csv')

In [12]:
testt=get_revenue_values(test2)
testt.to_csv('~/Downloads/Resi_ELRP_GroupB.csv')